In [1]:
import numpy as np
import pandas as pd
import os
import nibabel as nib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re

In [2]:
# The directory where our MRI .nii files are located!!!!
mri_image_dir = '../ADNI_RESIZED2' 

# Use os.listdir -->to get all file names in this directory
#Don't get the name one by one (which is what I did before
mri_image_paths = [os.path.join(mri_image_dir, fname) for fname in os.listdir(mri_image_dir) if fname.endswith('.nii')]

#print("Current working directory:", os.getcwd())
#print("Contents of the MRI directory:", os.listdir(mri_image_dir))
#print("Number of MRI files:", len(mri_image_paths))
#print("First 10 paths:", mri_image_paths[:10])

In [3]:
# 3D CNN architecture
def create_3d_cnn_model(width, height, depth, num_classes):
    inputShape = (width, height, depth, 1) 
    inputs = tf.keras.Input(inputShape)
    
    x = tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [4]:
# Load and preprocess an MRI image
def preprocess_and_extract_features_batch(file_paths, model):
    batch_data = []
    for file_path in file_paths:
        # Load the MRI image (.nii file) using nibabel
        image_data = nib.load(file_path)
        data = image_data.get_fdata()
        # Normalize the data and preprocess
        data = (data - np.min(data)) / (np.max(data) - np.min(data))
        data = np.resize(data, (64, 64, 64))  # Resize
        batch_data.append(data)
    
    batch_data = np.array(batch_data)
    batch_data = np.expand_dims(batch_data, axis=-1) 

    # 3D CNN to extract features that we need
    features = model.predict(batch_data)
    return features.reshape((features.shape[0], -1))

In [5]:
model = create_3d_cnn_model(64, 64, 64, num_classes=2)  # num_classes=2 --> binary classification


In [6]:
# process images in batches
batch_size = 10
features_list = []

for i in range(0, len(mri_image_paths), batch_size):
    batch_paths = mri_image_paths[i:i + batch_size]
    features = preprocess_and_extract_features_batch(batch_paths, model)
    features_list.append(features)

# Concatenate all batches to get the full features array
features = np.concatenate(features_list, axis=0)

1/1 [==============================] - 0s 226ms/step


In [7]:
np.savetxt('../extracted_features.csv', features, delimiter=',')  

In [8]:
# Read
labels_df = pd.read_csv('../Labels.csv')
print(labels_df.head())

# Create a dictionary mapping  Image Data ID and label
labels_dict = dict(zip(labels_df['Image Data ID'], labels_df['Group']))
print(list(labels_dict.items())[:5])

# extract the Image Data ID from the file name
def extract_image_data_id(filename):
    match = re.match(r'.*_(I\d+)\.nii', filename)
    if match:
        return match.group(1)
    return None  # Return None if the pattern does not match

# Extract the Image Data ID from each filename
mri_image_ids = [extract_image_data_id(os.path.basename(path)) for path in mri_image_paths]
print(list(mri_image_ids)[:5])

# Debug
mri_image_paths = [os.path.join(mri_image_dir, fname) for fname in os.listdir(mri_image_dir) if fname.endswith('.nii')]
mri_image_labels = [labels_dict.get(extract_image_data_id(os.path.basename(path))) for path in mri_image_paths]
for path, label in zip(mri_image_paths[:5], mri_image_labels[:5]):
    print(f'File: {path} - Label: {label}')

# only use images for which we have labels
# a list for features and labels where labels are available
filtered_features = []
filtered_labels = []
for image_id, feature in zip(mri_image_ids, features):
    label = labels_dict.get(image_id)
    if label is not None:
        filtered_features.append(feature)
        filtered_labels.append(label)

# Convert 
filtered_features = np.array(filtered_features)
filtered_labels = np.array(filtered_labels)

# Debug
print("First 10 feature-label pairs:")
for i in range(min(10, len(filtered_features))):  
    print(f"Feature {i}:", filtered_features[i])
    print(f"Label {i}:", filtered_labels[i])

  Image Data ID     Subject Group Sex  Age  Visit Modality  \
0       I729725  941_S_5193   SMC   F   74     30      MRI   
1       I388033  941_S_5193   SMC   F   73     24      MRI   
2       I375627  941_S_5193   SMC   F   73     22      MRI   
3       I388034  941_S_5193   SMC   F   73     24      MRI   
4       I729737  941_S_5193   SMC   F   74     30      MRI   

          Description       Type   Acq Date Format  Downloaded  
0  MT1; GradWarp; N3m  Processed  4/02/2015  NiFTI         NaN  
1  MT1; GradWarp; N3m  Processed  8/19/2013  NiFTI         NaN  
2  MT1; GradWarp; N3m  Processed  5/29/2013  NiFTI         NaN  
3  MT1; GradWarp; N3m  Processed  8/19/2013  NiFTI         NaN  
4  MT1; GradWarp; N3m  Processed  4/02/2015  NiFTI         NaN  
[('I729725', 'SMC'), ('I388033', 'SMC'), ('I375627', 'SMC'), ('I388034', 'SMC'), ('I729737', 'SMC')]
['I71281', 'I32674', 'I67669', 'I67678', 'I74185']
File: ../ADNI_RESIZED2\ADNI_003_S_0931_MR_MPR__GradWarp__B1_Correction_Br_20070904172

In [9]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(filtered_features, filtered_labels, test_size=0.2, random_state=42)

In [10]:
# Train a Random Forest Classifier on the features extracted by the 3D CNN
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Save the predictions to a CSV file
#np.savetxt('predictions.csv', y_pred, delimiter=',')
np.savetxt('../predictions.csv', y_pred, fmt='%s', delimiter=',')


# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [11]:
# Save the classification report to a text file
with open('../classification_report.txt', 'w') as f:
    f.write(report)
# Save the model's accuracy to a text file
with open('../model_accuracy.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy}\n")

In [12]:
# Output the accuracy and classification report
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.2782608695652174
Classification Report:
              precision    recall  f1-score   support

          AD       0.15      0.18      0.17        22
          CN       0.23      0.25      0.24        40
         MCI       0.39      0.34      0.36        53

    accuracy                           0.28       115
   macro avg       0.26      0.26      0.26       115
weighted avg       0.29      0.28      0.28       115

